# Naive Bayesian Classifier
### Q1. Bayes Rule을 이해하고 Naive  Bayes classifier가 사용하는 사후 확률 계산 과정을 서술하세요.

- Bayes Rule:   
$P(w_i|x) = \frac{P(x|w_i)|P(w_i)}{P(x)} = \frac{P(x|w_i) P(w_i)}{\Sigma_j P(x|w_j)P(w_j)}$
  -
  - $P(x|w_i)\text{: 사후 확률, posterior}\\
P(x|w_i) \text{: 가능도/우도, likelihood}\\
P(w_i) \text{: 사전 확률, prior}\\
P(x) \text{: 증거, evidence}$

A1.

### Q2. Naive Bayes Classification 방법을 이용해서 다음 생성된 리뷰 데이터에 기반한 감정 분석을 해봅시다.

In [1]:
# pip install pandas
import pandas as pd
import re

In [2]:
# 리뷰 데이터 생성
data = {
    'review': [
        'I love this great product! It exceeded my expectations.',
        'The Worst purchase I have ever made. Completely useless.',
        'It is an average product, nothing special but not terrible either.',
        'Great service and who can help but love this design? Highly recommend!',
        'Terrible experience, I will never buy from this poor brand again.',
        'It’s acceptable, but I expected better service, not just an acceptable one.',
        'Absolutely wonderful! I am very satisfied with this great service.',
        'The quality is poor and it broke after one use. Terrible enough!',
        'Acceptable product for the price, but there are better options out there.',
        'Great quality and fast shipping with wonderful service! I love it'
    ],
    'sentiment': [
        'positive', 'negative', 'neutral', 'positive', 'negative',
        'neutral', 'positive', 'negative', 'neutral', 'positive',
    ]
}
df = pd.DataFrame(data)
df.head()

,review,sentiment
0,I love this great product! It exceeded my expe...,positive
1,The Worst purchase I have ever made. Completel...,negative
2,"It is an average product, nothing special but ...",neutral
3,Great service and who can help but love this d...,positive
4,"Terrible experience, I will never buy from thi...",negative


In [3]:
# 불용어 리스트 정의
stopwords = ['i', 'my', 'am', 'this', 'it', 'its', 'an', 'a', 'the', 'is', 'are', 'and', 'product', 'service']

In [4]:
# 텍스트 전처리 함수 정의
def preprocess_text(text):
    # 소문자로 변환
    text = text.lower()
    # 특수 기호 제거
    text = re.sub(r'[^a-z\s]', '', text)
    # 불용어 제거
    words = text.split()
    filtered_words = [word for word in words if word not in stopwords]
    return ' '.join(filtered_words)

# 모든 리뷰에 대해 전처리 수행
df['review'] = df['review'].apply(preprocess_text)

기본적인 데이터 전처리가 완료되었습니다!
이제부터 직접 나이브 베이지안 분류를 수행해 봅시다.  
우리가 분류하고자 하는 문장은 총 두가지 입니다.  
전처리가 완료되었다고 치고,   
첫번째 문장은 **'love, great, awesome'**,  
두번째 문장은 **'terrible, not, never'** 입니다.

사전 확률 $P(positive), P(negative), P(neutral)$을 구합니다.

In [5]:
# 사전 확률 구하는 코드를 작성해주세요.

def prior(sentiment):
  num = (df['sentiment'] == sentiment).sum()
  return num/len(df)

# 사전 확률 구하기
print(f'P(positive) = {prior("positive")}')
print(f'P(negative) = {prior("negative")}')
print(f'P(neutral) = {prior("neutral")}')

P(positive) = 0.4
P(negative) = 0.3
P(neutral) = 0.3


가능도를 구하기 위한 확률들을 계산합니다.  
예: 첫번째 문장 분류를 위해서는, $P(love|positive), P(great|positive), P(awesome|positive)\\
P(love|negative), P(great|negative), P(awesome|negative)\\
P(love|neutral), P(great|neutral), P(great|neutral)$를 구합니다.

이 때 CountVectorizer를 사용하여 도출한 단어 벡터를 활용하면 확률들을 간편하게 구할 수 있습니다.  
참고: https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html

In [6]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
review_array = vectorizer.fit_transform(df['review']).toarray()
review_array

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0,
        0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
        0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
        0, 0, 0, 1, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
        1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
        0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
        1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
        1, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 2, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 

In [7]:
vectorizer.get_feature_names_out()

array(['absolutely', 'acceptable', 'after', 'again', 'average', 'better',
       'brand', 'broke', 'but', 'buy', 'can', 'completely', 'design',
       'either', 'enough', 'ever', 'exceeded', 'expectations', 'expected',
       'experience', 'fast', 'for', 'from', 'great', 'have', 'help',
       'highly', 'just', 'love', 'made', 'never', 'not', 'nothing', 'one',
       'options', 'out', 'poor', 'price', 'purchase', 'quality',
       'recommend', 'satisfied', 'shipping', 'special', 'terrible',
       'there', 'use', 'useless', 'very', 'who', 'will', 'with',
       'wonderful', 'worst'], dtype=object)

In [8]:
vectorizer.vocabulary_

{'love': 28,
 'great': 23,
 'exceeded': 16,
 'expectations': 17,
 'worst': 53,
 'purchase': 38,
 'have': 24,
 'ever': 15,
 'made': 29,
 'completely': 11,
 'useless': 47,
 'average': 4,
 'nothing': 32,
 'special': 43,
 'but': 8,
 'not': 31,
 'terrible': 44,
 'either': 13,
 'who': 49,
 'can': 10,
 'help': 25,
 'design': 12,
 'highly': 26,
 'recommend': 40,
 'experience': 19,
 'will': 50,
 'never': 30,
 'buy': 9,
 'from': 22,
 'poor': 36,
 'brand': 6,
 'again': 3,
 'acceptable': 1,
 'expected': 18,
 'better': 5,
 'just': 27,
 'one': 33,
 'absolutely': 0,
 'wonderful': 52,
 'very': 48,
 'satisfied': 41,
 'with': 51,
 'quality': 39,
 'broke': 7,
 'after': 2,
 'use': 46,
 'enough': 14,
 'for': 21,
 'price': 37,
 'there': 45,
 'options': 34,
 'out': 35,
 'fast': 20,
 'shipping': 42}

In [9]:
frequency_matrix = pd.DataFrame(review_array, columns = vectorizer.get_feature_names_out())
frequency_matrix = pd.concat([df['sentiment'], frequency_matrix], axis=1)
frequency_matrix

,sentiment,absolutely,acceptable,after,again,average,better,brand,broke,but,...,terrible,there,use,useless,very,who,will,with,wonderful,worst
0,positive,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,negative,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
2,neutral,0,0,0,0,1,0,0,0,1,...,1,0,0,0,0,0,0,0,0,0
3,positive,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
4,negative,0,0,0,1,0,0,1,0,0,...,1,0,0,0,0,0,1,0,0,0
5,neutral,0,2,0,0,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
6,positive,1,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,1,1,0
7,negative,0,0,1,0,0,0,0,1,0,...,1,0,1,0,0,0,0,0,0,0
8,neutral,0,1,0,0,0,1,0,0,1,...,0,2,0,0,0,0,0,0,0,0
9,positive,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0


In [14]:
# 위와 같이 조건부 확률을 구하는 코드를 작성해주세요
def calculate_likelihood(word, sentiment):
    # 해당 감성의 데이터만 선택

    review_count = (prior(sentiment) * len(df)) + 1 # 특정 감정의 리뷰 수 + 라플라스 스무딩 계수
    reviews = frequency_matrix[frequency_matrix['sentiment'] == sentiment] # 특정 감정으로 필터링된 리뷰 df

    word_count_in_sentiments = reviews[word].sum() + 1 if word in reviews.columns else 1 # 필터링된 df에서 특정 단어의 빈도 합산(+ 라플라스 스무딩 계수 추가)

    return word_count_in_sentiments / review_count

# 예시: 'positive' 감성일 때 'absolutely' 단어가 존재할 확률
for i in vectorizer.vocabulary_.keys():
  prob = calculate_likelihood(i, 'positive')
  print(f"'{i}' 단어가 존재할 때 'positive' 감성일 확률: {prob:.2f}")






'love' 단어가 존재할 때 'positive' 감성일 확률: 0.80
'great' 단어가 존재할 때 'positive' 감성일 확률: 1.00
'exceeded' 단어가 존재할 때 'positive' 감성일 확률: 0.40
'expectations' 단어가 존재할 때 'positive' 감성일 확률: 0.40
'worst' 단어가 존재할 때 'positive' 감성일 확률: 0.20
'purchase' 단어가 존재할 때 'positive' 감성일 확률: 0.20
'have' 단어가 존재할 때 'positive' 감성일 확률: 0.20
'ever' 단어가 존재할 때 'positive' 감성일 확률: 0.20
'made' 단어가 존재할 때 'positive' 감성일 확률: 0.20
'completely' 단어가 존재할 때 'positive' 감성일 확률: 0.20
'useless' 단어가 존재할 때 'positive' 감성일 확률: 0.20
'average' 단어가 존재할 때 'positive' 감성일 확률: 0.20
'nothing' 단어가 존재할 때 'positive' 감성일 확률: 0.20
'special' 단어가 존재할 때 'positive' 감성일 확률: 0.20
'but' 단어가 존재할 때 'positive' 감성일 확률: 0.40
'not' 단어가 존재할 때 'positive' 감성일 확률: 0.20
'terrible' 단어가 존재할 때 'positive' 감성일 확률: 0.20
'either' 단어가 존재할 때 'positive' 감성일 확률: 0.20
'who' 단어가 존재할 때 'positive' 감성일 확률: 0.40
'can' 단어가 존재할 때 'positive' 감성일 확률: 0.40
'help' 단어가 존재할 때 'positive' 감성일 확률: 0.40
'design' 단어가 존재할 때 'positive' 감성일 확률: 0.40
'highly' 단어가 존재할 때 'positive' 감성일 확률: 0.40
'recommend' 단어가

독립성 가정을 이용하여 가능도(likelihood)를 구합니다.  
첫번째 문장 예시: $P(love, great, awesome|positive), P(love, great, awesome|negative), P(love, great, awesome|neutral)$

In [16]:
# 가능도 구하는 코드를 작성해주세요.
def likelihood(words, sentiment):
  likelihood = 1.0
  for word in words:
    likelihood *= calculate_likelihood(word, sentiment)
  return likelihood



words1 = ['love', 'great', 'awesome']
words2 = ['terrible', 'not', 'never']


ll1_positive = likelihood(words1, "positive")
ll1_negative = likelihood(words1, "negative")
ll1_neutral = likelihood(words1, "neutral")

ll2_positive = likelihood(words2, "positive")
ll2_negative = likelihood(words2, "negative")
ll2_neutral = likelihood(words2, "neutral")

print('words -> pos, neg, neu')
print('love, great, awesome ->[',ll1_positive, ll1_negative, ll1_neutral, ']' )

print('terrible, not, never ->[',ll2_positive, ll2_negative, ll2_neutral, ']' )



words -> pos, neg, neu
love, great, awesome ->[ 0.16000000000000003 0.015625 0.015625 ]
terrible, not, never ->[ 0.008000000000000002 0.09375 0.09375 ]


위에서 구한 사전 확률과 가능도를 이용하여 타겟 문장이 positive, negative, neutral일 확률을 구하고 최종적으로 어떤 감성일지 분석해봅니다.

In [ ]:
import numpy as np

target_review1 = df['review'][0]
target_review2 = df['review'][1]


prior_positive = prior("positive")
prior_negative = prior("negative")
prior_neutral = prior("neutral")


# 최종 확률 구하는 코드를 작성해주세요.
# 첫번째 문장
# P(positive|target_review1)

# P(negative|target_review1)

# P(neutral|target_review1)

# 두번째 문장
# P(positive|target_review2)

# P(negative|target_review2)

# P(neutral|target_review2)




P(positive|target_review1): 0.0
P(negative|target_review1): 0.0
P(neutral|target_review1): 0.0


P(positive|target_review2): 0.0
P(negative|target_review2): 4.340277777777777e-05
P(neutral|target_review2): 0.0


A2-1.   
Target review1의 분류 결과:   
Target review2의 분류 결과:

Q2-2. 나이브 베이지안 기반 확률을 구하는 과정에서 어떤 문제점을 발견할 수 있었나요? 그리고 그 문제를 해결하기 위한 방법에 대해 간략하게 조사 및 서술해 주세요. (힌트: Laplace smoothing)



A2-2. awesome이라는 단어가 없어서 구할 수 없었다. 그래서 라플라스 스무딩을 사용해서 없는 단어에 대해서 사용할 수 있도록 계수를 설정한다. 여기서는 1로 설정해서 계산했다.